<a href="https://colab.research.google.com/github/Gustavo0501/Trabalho_Grafos/blob/main/Etapa3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Integrantes:
# Alexandre Marques Spinola
# Gustavo do Carmo Resende




In [ ]:
import os
import math
import sys
import copy
import time


class Rota:
    """Representa a rota de um único veículo."""
    def __init__(self, id_rota, deposito):
        self.id_rota = id_rota
        self.deposito = deposito
        self.carga_total = 0
        self.custo_total = 0
        self.sequencia_visitas = [f"(D 0,{deposito},{deposito})"] # Inicia no depósito
        self.servicos_atendidos = []
        self.posicao_atual = deposito

    def adicionar_servico(self, servico, custo_deslocamento, ponto_entrada, ponto_saida):
        """Adiciona um serviço à rota e atualiza os totais."""
        self.carga_total += servico['demanda']
        self.custo_total += custo_deslocamento + servico['custo']
        self.posicao_atual = ponto_saida
        self.sequencia_visitas.append(f"(S {servico['id']},{ponto_entrada},{ponto_saida})")
        self.servicos_atendidos.append(servico)

    def finalizar_rota(self, distancias):
        """Adiciona o custo de retorno ao depósito e finaliza a rota."""
        custo_retorno = distancias[self.posicao_atual][self.deposito]
        self.custo_total += custo_retorno
        self.sequencia_visitas.append(f"(D 0,{self.deposito},{self.deposito})")

    def to_string(self):
        """Formata a rota para o arquivo de solução."""
        return f" 0 1 {self.id_rota} {self.carga_total} {int(self.custo_total)} {len(self.sequencia_visitas)} {' '.join(self.sequencia_visitas)}"

class Solucao:
    """Representa a solução completa para uma instância."""
    def __init__(self):
        self.rotas = []
        self.custo_total = 0

    def adicionar_rota(self, rota):
        self.rotas.append(rota)
        self.custo_total += rota.custo_total

    def salvar_em_arquivo(self, nome_arquivo_instancia, pasta_saida, tempo_total_ns, tempo_encontrar_ns):
        """Salva a solução formatada, incluindo os dois tempos de execução medidos."""
        os.makedirs(pasta_saida, exist_ok=True)
        caminho_saida = os.path.join(pasta_saida, f"sol-{os.path.splitext(nome_arquivo_instancia)[0]}.dat")

        with open(caminho_saida, 'w') as f:
            custo_final = sum(r.custo_total for r in self.rotas)
            f.write(f"{int(custo_final)}\n")
            f.write(f"{len(self.rotas)}\n")

            # Escreve os dois valores de tempo distintos
            f.write(f"{tempo_total_ns}\n")      # Tempo total de execução (Etapa 2 + Etapa 3)
            f.write(f"{tempo_encontrar_ns}\n")  # Tempo para encontrar a primeira solução (só Etapa 2)

            for rota in self.rotas:
                f.write(f"{rota.to_string()}\n")
        print(f"Solução salva em: {caminho_saida}")

class Grafo:
    """Lê e armazena os dados de uma instância do problema."""
    def __init__(self, caminho_arquivo):
        self.nome_arquivo = os.path.basename(caminho_arquivo)
        self.vertices = 0
        self.deposito = None
        self.capacidade = 0
        self.max_veiculos = -1
        self.arestas = []
        self.arcos = []
        self.servicos_requeridos = []
        self.distancias = []
        self.predecessores = []

        self._ler_arquivo_instancia(caminho_arquivo)
        self._calcular_caminhos_curtos()

    def _ler_arquivo_instancia(self, caminho_arquivo):
        """Lê o arquivo de instância e preenche os atributos do grafo de forma robusta."""
        with open(caminho_arquivo, 'r') as f:
            linhas = [linha.strip() for linha in f if linha.strip() and not linha.lower().startswith("based")]

        secao = None
        id_servico_counter = 1

        for linha in linhas:
            if linha == "-1":
                continue

            if ':' in linha:
                chave, valor = [p.strip() for p in linha.split(':')]
                if chave == 'Depot Node': self.deposito = int(valor)
                elif chave == '#Nodes': self.vertices = int(valor)
                elif chave == 'Capacity': self.capacidade = int(valor)
                elif chave == '#Vehicles': self.max_veiculos = int(valor)
            elif linha.startswith('ReN.'): secao = 'ReN'
            elif linha.startswith('ReE.'): secao = 'ReE'
            elif linha.startswith('ReA.'): secao = 'ReA'
            elif linha.startswith('EDGE'): secao = 'EDGE'
            elif linha.startswith('ARC'): secao = 'ARC'
            else:
                partes = linha.split()
                if not partes: continue

                try:
                    if secao == 'ReN':
                        no = int(partes[0][1:])
                        demanda = int(partes[1])
                        custo_servico = int(partes[2])
                        self.servicos_requeridos.append({'id': id_servico_counter, 'tipo': 'V', 'nos': (no, no), 'custo': custo_servico, 'demanda': demanda})
                        id_servico_counter += 1
                    elif secao == 'ReE':
                        u, v = int(partes[1]), int(partes[2])
                        custo_servico = int(partes[3])
                        demanda = int(partes[4])
                        self.servicos_requeridos.append({'id': id_servico_counter, 'tipo': 'E', 'nos': (u, v), 'custo': custo_servico, 'demanda': demanda})
                        self.arestas.append((u, v, custo_servico))
                        id_servico_counter += 1
                    elif secao == 'ReA':
                        u, v = int(partes[1]), int(partes[2])
                        custo_servico = int(partes[3])
                        demanda = int(partes[4])
                        self.servicos_requeridos.append({'id': id_servico_counter, 'tipo': 'A', 'nos': (u, v), 'custo': custo_servico, 'demanda': demanda})
                        self.arcos.append((u, v, custo_servico))
                        id_servico_counter += 1
                    elif secao == 'EDGE':
                        u, v, custo = int(partes[1]), int(partes[2]), int(partes[3])
                        self.arestas.append((u, v, custo))
                    elif secao == 'ARC':
                        u, v, custo = int(partes[1]), int(partes[2]), int(partes[3])
                        self.arcos.append((u, v, custo))
                except (ValueError, IndexError):
                    continue

    def _calcular_caminhos_curtos(self):
        V = self.vertices
        self.distancias = [[math.inf] * (V + 1) for _ in range(V + 1)]
        self.predecessores = [[-1] * (V + 1) for _ in range(V + 1)]

        for i in range(1, V + 1):
            self.distancias[i][i] = 0

        for u, v, custo in self.arestas:
            if custo < self.distancias[u][v]:
                self.distancias[u][v] = self.distancias[v][u] = custo
                self.predecessores[u][v] = u
                self.predecessores[v][u] = v

        for u, v, custo in self.arcos:
            if custo < self.distancias[u][v]:
                self.distancias[u][v] = custo
                self.predecessores[u][v] = u

        for k in range(1, V + 1):
            for i in range(1, V + 1):
                for j in range(1, V + 1):
                    if self.distancias[i][k] + self.distancias[k][j] < self.distancias[i][j]:
                        self.distancias[i][j] = self.distancias[i][k] + self.distancias[k][j]
                        self.predecessores[i][j] = self.predecessores[k][j]
        print(f"Matriz de distâncias calculada para {self.nome_arquivo}.")

class PathScanning:
    """Implementa o algoritmo construtivo Path Scanning."""
    def __init__(self, grafo):
        self.grafo = grafo

    def executar(self):
        """Executa o algoritmo e retorna um objeto Solucao, respeitando o limite de veículos."""
        solucao = Solucao()
        servicos_nao_atendidos = set(s['id'] for s in self.grafo.servicos_requeridos)
        servicos_map = {s['id']: s for s in self.grafo.servicos_requeridos}
        rota_id = 1

        # O limite de veículos só é considerado se for um valor positivo
        limite_veiculos = self.grafo.max_veiculos if self.grafo.max_veiculos > 0 else float('inf')

        while servicos_nao_atendidos:
            # VERIFICA A RESTRIÇÃO DE VEÍCULOS ANTES DE CRIAR UMA NOVA ROTA
            if rota_id > limite_veiculos:
                print(f"AVISO: Limite de {int(limite_veiculos)} veículos atingido, mas ainda há {len(servicos_nao_atendidos)} serviços não atendidos.")
                print("       A heurística não conseguiu encontrar uma solução viável com a frota disponível.")
                break # Interrompe a criação de novas rotas

            rota_atual = Rota(rota_id, self.grafo.deposito)

            while True:
                candidatos = []
                for sid in servicos_nao_atendidos:
                    servico = servicos_map[sid]
                    if rota_atual.carga_total + servico['demanda'] <= self.grafo.capacidade:
                        u, v = servico['nos']
                        dist_para_u = self.grafo.distancias[rota_atual.posicao_atual][u]
                        dist_para_v = self.grafo.distancias[rota_atual.posicao_atual][v]

                        if servico['tipo'] == 'E' and dist_para_v < dist_para_u:
                            custo_deslocamento = dist_para_v
                            ponto_entrada, ponto_saida = v, u
                        else:
                            custo_deslocamento = dist_para_u
                            ponto_entrada, ponto_saida = u, v

                        candidatos.append((custo_deslocamento, servico, ponto_entrada, ponto_saida))

                if not candidatos:
                    break

                candidatos.sort(key=lambda x: x[0])
                melhor_custo, melhor_servico, entrada, saida = candidatos[0]

                rota_atual.adicionar_servico(melhor_servico, melhor_custo, entrada, saida)
                servicos_nao_atendidos.remove(melhor_servico['id'])

            rota_atual.finalizar_rota(self.grafo.distancias)
            solucao.adicionar_rota(rota_atual)
            rota_id += 1

        return solucao


class BuscaLocal:
    """
    Aplica algoritmos de busca local para refinar uma solução existente.
    """
    def __init__(self, solucao_inicial, grafo):
        self.solucao_atual = copy.deepcopy(solucao_inicial)
        self.grafo = grafo
        self.servicos_map = {s['id']: s for s in self.grafo.servicos_requeridos}

    def executar(self):
        print("\n--- Iniciando Busca Local para refinar a solução ---")
        melhoria_encontrada = True
        iteracao = 0
        while melhoria_encontrada:
            iteracao += 1
            print(f"Iteração {iteracao} da Busca Local... Custo atual: {int(self.solucao_atual.custo_total)}")
            melhoria_encontrada = self._tentar_transferencia()
            if not melhoria_encontrada:
                melhoria_encontrada = self._tentar_re_insercao_intra_rota()

        print(f"Busca Local finalizada. Custo final: {int(self.solucao_atual.custo_total)}")
        return self.solucao_atual

    def _recalcular_custo_e_sequencia_rota(self, lista_servicos):
        custo = 0
        posicao_atual = self.grafo.deposito
        sequencia_visitas = [f"(D 0,{self.grafo.deposito},{self.grafo.deposito})"]

        for servico in lista_servicos:
            ponto_entrada, ponto_saida = servico['nos']
            custo_deslocamento = self.grafo.distancias[posicao_atual][ponto_entrada]
            custo += custo_deslocamento + servico['custo']
            posicao_atual = ponto_saida
            sequencia_visitas.append(f"(S {servico['id']},{ponto_entrada},{ponto_saida})")

        custo += self.grafo.distancias[posicao_atual][self.grafo.deposito]
        sequencia_visitas.append(f"(D 0,{self.grafo.deposito},{self.grafo.deposito})")

        return custo, sequencia_visitas

    def _tentar_transferencia(self):
        for rota_origem in self.solucao_atual.rotas:
            for i in range(len(rota_origem.servicos_atendidos) - 1, -1, -1):
                servico_a_mover = rota_origem.servicos_atendidos[i]

                for rota_destino in self.solucao_atual.rotas:
                    if rota_origem.id_rota == rota_destino.id_rota:
                        continue

                    if rota_destino.carga_total + servico_a_mover['demanda'] > self.grafo.capacidade:
                        continue

                    custo_antigo_combinado = rota_origem.custo_total + rota_destino.custo_total
                    servicos_origem_temp = rota_origem.servicos_atendidos[:i] + rota_origem.servicos_atendidos[i+1:]
                    novo_custo_origem, _ = self._recalcular_custo_e_sequencia_rota(servicos_origem_temp)

                    melhor_custo_destino = float('inf')
                    melhor_posicao_insercao = -1

                    for j in range(len(rota_destino.servicos_atendidos) + 1):
                        servicos_destino_temp = rota_destino.servicos_atendidos[:j] + [servico_a_mover] + rota_destino.servicos_atendidos[j:]
                        custo_teste, _ = self._recalcular_custo_e_sequencia_rota(servicos_destino_temp)
                        if custo_teste < melhor_custo_destino:
                            melhor_custo_destino = custo_teste
                            melhor_posicao_insercao = j

                    if novo_custo_origem + melhor_custo_destino < custo_antigo_combinado:
                        rota_origem.servicos_atendidos.pop(i)
                        rota_origem.carga_total -= servico_a_mover['demanda']
                        rota_origem.custo_total, rota_origem.sequencia_visitas = self._recalcular_custo_e_sequencia_rota(rota_origem.servicos_atendidos)

                        rota_destino.servicos_atendidos.insert(melhor_posicao_insercao, servico_a_mover)
                        rota_destino.carga_total += servico_a_mover['demanda']
                        rota_destino.custo_total, rota_destino.sequencia_visitas = self._recalcular_custo_e_sequencia_rota(rota_destino.servicos_atendidos)

                        self.solucao_atual.custo_total = sum(r.custo_total for r in self.solucao_atual.rotas)

                        print(f"  Melhoria (Transferência): Serviço {servico_a_mover['id']} movido da Rota {rota_origem.id_rota} para Rota {rota_destino.id_rota}.")
                        return True
        return False

    def _tentar_re_insercao_intra_rota(self):
        for rota in self.solucao_atual.rotas:
            if len(rota.servicos_atendidos) < 2:
                continue

            for i in range(len(rota.servicos_atendidos)):
                servico_a_mover = rota.servicos_atendidos[i]
                servicos_temp = rota.servicos_atendidos[:i] + rota.servicos_atendidos[i+1:]
                custo_original = rota.custo_total

                for j in range(len(servicos_temp) + 1):
                    if i == j:
                        continue

                    servicos_teste = servicos_temp[:j] + [servico_a_mover] + servicos_temp[j:]
                    novo_custo, _ = self._recalcular_custo_e_sequencia_rota(servicos_teste)

                    if novo_custo < custo_original:
                        rota.servicos_atendidos = servicos_teste
                        rota.custo_total, rota.sequencia_visitas = self._recalcular_custo_e_sequencia_rota(rota.servicos_atendidos)
                        self.solucao_atual.custo_total = sum(r.custo_total for r in self.solucao_atual.rotas)

                        print(f"  Melhoria (Re-inserção): Serviço {servico_a_mover['id']} reposicionado na Rota {rota.id_rota}.")
                        return True
        return False

def main():
    """Função principal para executar o processo completo para todas as instâncias."""
    pasta_entrada = 'MCGRP/'
    pasta_saida = 'G13/'

    if not os.path.isdir(pasta_entrada):
        print(f"Erro: A pasta de entrada '{pasta_entrada}' não foi encontrada.")
        sys.exit(1)

    arquivos_instancia = [f for f in os.listdir(pasta_entrada) if f.endswith('.dat')]

    for nome_arquivo in arquivos_instancia:
        print(f"\n--- Processando instância: {nome_arquivo} ---")
        try:
            inicio_processo_ns = time.perf_counter_ns()

            caminho_completo = os.path.join(pasta_entrada, nome_arquivo)

            grafo = Grafo(caminho_completo)

            algoritmo_construtivo = PathScanning(grafo)
            solucao_inicial = algoritmo_construtivo.executar()

            # Captura o tempo para encontrar a primeira solução (fim da Etapa 2)
            fim_etapa2_ns = time.perf_counter_ns()
            tempo_para_encontrar_ns = fim_etapa2_ns - inicio_processo_ns

            # Verifica se a solução inicial é factível (cobriu todos os serviços)
            servicos_atendidos_count = sum(len(r.servicos_atendidos) for r in solucao_inicial.rotas)
            if servicos_atendidos_count < len(grafo.servicos_requeridos):
                print("A solução inicial é infactível (não cobre todos os serviços devido ao limite de veículos).")
                print("Salvando a solução parcial gerada pela Etapa 2...")
                solucao_final = solucao_inicial
            else:
                print(f"Custo da solução inicial (Path Scanning): {int(solucao_inicial.custo_total)}")
                # Etapa 3: Refinamento com Busca Local
                busca_local = BuscaLocal(solucao_inicial, grafo)
                solucao_melhorada = busca_local.executar()
                solucao_final = solucao_melhorada

            # Captura o tempo total de execução
            fim_total_ns = time.perf_counter_ns()
            tempo_total_ns = fim_total_ns - inicio_processo_ns

            # Salva a solução final, passando os dois tempos medidos
            solucao_final.salvar_em_arquivo(nome_arquivo, pasta_saida, tempo_total_ns, tempo_para_encontrar_ns)

        except Exception as e:
            print(f"Ocorreu um erro ao processar {nome_arquivo}: {e}")
            import traceback
            traceback.print_exc()

if __name__ == "__main__":
    main()